<a href="https://colab.research.google.com/github/lIlIlIIIlIIl/Cube-Solver/blob/main/Cube_Color_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ultralystics 설치

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install -U ultralytics

# Import

In [ ]:
!pip install onnx
!pip install onnxruntime-gpu

In [ ]:
import os
import cv2
import numpy as np

from ultralytics.utils.plotting import Annotator, colors
from ultralytics import YOLO

import torch
from PIL import Image
from torchvision import transforms

# Cube Detector Training

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")

# Train the model
train_results = model.train(
    data="https://universe.roboflow.com/ds/f1PhdDnpq6?key=ziMPHpXihd",  # path to dataset
    epochs=100,  # number of training epochs
    imgsz=640,  # training image size
    device="0",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

In [ ]:
# Evaluate model performance on the validation set
metrics = model.val() # no arguments needed, dataset and settings remembered

# Export the model to ONNX format
path = model.export(format="onnx")  # return path to exported model

# Color Detector Training(데이터셋 확보 X)

In [ ]:
model = YOLO("yolo11n.pt")

# Train the model
train_results = model.train(
    data="https://universe.roboflow.com/ds/g1SAsAJD1a?key=AJZ5k2p9Cz",  # path to dataset YAML
    epochs=100,  # number of training epochs
    imgsz=640,  # training image size
    device="0",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

In [ ]:
# Evaluate model performance on the validation set
metrics = model.val() # no arguments needed, dataset and settings remembered
path = model.export(format="onnx")

# Cube Detector

In [ ]:
def CubeDetector(img, model_onnx):
    # img = "/content/sample_image.png"

    model = YOLO(model_onnx, task='detect')
    results = model.predict(img, iou=0.45, device="cpu", max_det=1) # return a list of Results objects
    results[0].show()

    crop_dir_name = "detected_cube"
    names = model.names
    boxes = results[0].boxes.xyxy.cpu().tolist()
    clss = results[0].boxes.cls.cpu().tolist()

    img = np.array(Image.open(img))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # PIL은 RGB, OpenCV는 BGR이므로 색 변환

    annotator = Annotator(img, line_width=2, example=names)

    if boxes is not None:
        for box, cls in zip(boxes, clss):
            annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

            crop_obj = img[int(box[1]) : int(box[3]), int(box[0]) : int(box[2])]

            cv2.imwrite(os.path.join(crop_dir_name + ".png"), crop_obj)

    return crop_dir_name + ".png"

# Color Detector

In [ ]:
def ColorDetector(img, model_onnx):
    # img = "/content/detected_cube.png"

    model = YOLO(model_onnx, task="detect")
    results = model.predict(img, iou=0.45, device='cpu') # return a list of Results objects
    results[0].show()